# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

Assignment 2 Nov 1 2200h

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [1]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.test', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))

adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [ ]:
# Create X and Y dataframes
X = adult_dt.drop('income', axis=1)
Y = adult_dt['income']

# Split data into training and testing sets 70:30 with a random state of 42
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, 
    test_size=0.3,   
    random_state=42   
)

# Print shapes of datasets
print("Training set shapes:")
print(f"X_train: {X_train.shape}")
print(f"Y_train: {Y_train.shape}")
print("\nTesting set shapes:")
print(f"X_test: {X_test.shape}")
print(f"Y_test: {Y_test.shape}")

# Print first few rows of X_train and Y_train
print("\nFirst few rows of X_train:")
print(X_train.head())
print("\nFirst few rows of Y_train:")
print(Y_train.head())

Training set shapes:
X_train: (22792, 14)
Y_train: (22792,)

Testing set shapes:
X_test: (9769, 14)
Y_test: (9769,)

First few rows of X_train:
       age          workclass  fnlwgt      education  education-num  \
19749   34   Self-emp-not-inc   56460        HS-grad              9   
1216    48   Self-emp-not-inc  243631   Some-college             10   
27962   23          State-gov   56402   Some-college             10   
23077   56          Local-gov  255406        HS-grad              9   
10180   17            Private  297246           11th              7   

            marital-status        occupation    relationship  \
19749   Married-civ-spouse   Farming-fishing            Wife   
1216    Married-civ-spouse      Craft-repair         Husband   
27962   Married-civ-spouse    Prof-specialty         Husband   
23077             Divorced   Exec-managerial   Not-in-family   
10180        Never-married   Priv-house-serv       Own-child   

                      race      sex  capital

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

# What is the random state of the splitting function?
The random state in the splitting function is a number that controls how the data is split into the training and testing sets. When this function is applied, the data is randomly mixed up before it is split. By setting the random_state to a specific number, you make sure that every time you run the code, the same split of the data is produced.  

Why is it useful?
Using a fixed random_state is helpful for the following reasons:

1/ Consistency in that it ensures that you get the same results every time you run your code. This is important when you want to compare how different models perform under the same conditions.

2/ Debugging: When working on models, having the same data split helps to identify changes made to improve the model rather than just changing the data it trains on.

3/ Collaboration - Using a fixed random_state allows allows for improved collaboration in that the random state is set across collaborators, which is important for teamwork and verifying each other's work.

4/ Research and Publication: In research, being able to repeat experiments and get the same results is very important. Using a fixed random_state helps others replicate your work and confirm your findings.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [22]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder

# Identify numerical and categorical columns
numerical_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 
                     'capital-loss', 'hours-per-week']
categorical_features = ['workclass', 'education', 'marital-status', 'occupation',
                       'relationship', 'race', 'sex', 'native-country']

# Create numerical pipeline
numerical_pipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

# Create categorical pipeline
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

# Create column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ])

# Print structure of preprocessor
print("Column Transformer Structure:")
print(preprocessor)

# Print feature lists to verify correct column assignment
print("\nNumerical Features:")
print(numerical_features)
print("\nCategorical Features:")
print(categorical_features)


Column Transformer Structure:
ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('scaler', RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'))]),
              

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Create pipeline
pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier())
])

# Print pipeline structure  
print("Pipeline Structure:")
print(pipe)

Pipeline Structure:
Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat',
                                                  Pipelin

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [24]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, log_loss, roc_auc_score
import numpy as np

# Define scoring metrics
scoring = {
    'neg_log_loss': 'neg_log_loss',
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}

# Perform cross-validation
cv_results = cross_validate(
    pipe,
    X_train,
    Y_train,
    scoring=scoring,
    cv=5,                # 5 folds
    return_train_score=True,  # Get both training and validation scores
    n_jobs=-1           # Use all available cores
)

# Print results
print("Cross-validation results:")
for metric in scoring.keys():
    train_scores = cv_results[f'train_{metric}']
    test_scores = cv_results[f'test_{metric}']
    
    print(f"\n{metric}:")
    print(f"  Training:   mean = {train_scores.mean():.3f} (+/- {train_scores.std() * 2:.3f})")
    print(f"  Validation: mean = {test_scores.mean():.3f} (+/- {test_scores.std() * 2:.3f})")

Cross-validation results:

neg_log_loss:
  Training:   mean = -0.082 (+/- 0.001)
  Validation: mean = -0.378 (+/- 0.045)

roc_auc:
  Training:   mean = 1.000 (+/- 0.000)
  Validation: mean = 0.903 (+/- 0.004)

accuracy:
  Training:   mean = 1.000 (+/- 0.000)
  Validation: mean = 0.853 (+/- 0.007)

balanced_accuracy:
  Training:   mean = 1.000 (+/- 0.000)
  Validation: mean = 0.776 (+/- 0.006)


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [18]:
# Create DataFrame from cross-validation results
results_df = pd.DataFrame({
    'train_neg_log_loss': cv_results['train_neg_log_loss'],
    'test_neg_log_loss': cv_results['test_neg_log_loss'],
    'train_roc_auc': cv_results['train_roc_auc'],
    'test_roc_auc': cv_results['test_roc_auc'],
    'train_accuracy': cv_results['train_accuracy'],
    'test_accuracy': cv_results['test_accuracy'],
    'train_balanced_accuracy': cv_results['train_balanced_accuracy'],
    'test_balanced_accuracy': cv_results['test_balanced_accuracy']
})

# Add fold number column
results_df.index.name = 'Fold'
results_df.index += 1  # Start fold numbering at 1

# Sort by test neg_log_loss
results_df_sorted = results_df.sort_values('test_neg_log_loss', ascending=True)

# Display sorted results
print("Cross-validation results sorted by test negative log loss:")
print(results_df_sorted)

Cross-validation results sorted by test negative log loss:
      train_neg_log_loss  test_neg_log_loss  train_roc_auc  test_roc_auc  \
Fold                                                                       
3              -0.081471          -0.391009            1.0      0.900593   
4              -0.082184          -0.378408            1.0      0.906683   
5              -0.081660          -0.373080            1.0      0.902779   
2              -0.081000          -0.361883            1.0      0.902086   
1              -0.081721          -0.336490            1.0      0.904355   

      train_accuracy  test_accuracy  train_balanced_accuracy  \
Fold                                                           
3           1.000000       0.849276                 1.000000   
4           1.000000       0.860465                 1.000000   
5           0.999945       0.856297                 0.999887   
2           1.000000       0.846458                 1.000000   
1           1.000000    

Calculate the mean of each metric. 

In [19]:
# Calculate means for all metrics
mean_scores = results_df.mean()

# Format display of means
print("Mean scores across all folds:")
print("\nTraining Metrics:")
print("-----------------")
for metric in mean_scores.index:
    if metric.startswith('train_'):
        print(f"{metric[6:]:20} {mean_scores[metric]:.3f}")

print("\nValidation Metrics:")
print("------------------")
for metric in mean_scores.index:
    if metric.startswith('test_'):
        print(f"{metric[5:]:20} {mean_scores[metric]:.3f}")

Mean scores across all folds:

Training Metrics:
-----------------
neg_log_loss         -0.082
roc_auc              1.000
accuracy             1.000
balanced_accuracy    1.000

Validation Metrics:
------------------
neg_log_loss         -0.368
roc_auc              0.903
accuracy             0.852
balanced_accuracy    0.774


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [21]:
# Make predictions on test set
Y_pred_proba = pipe.predict_proba(X_test)
Y_pred = pipe.predict(X_test)

# Calculate performance metrics
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Calculate metrics
neg_log_loss_value = log_loss(Y_test, Y_pred_proba)
roc_auc_value = roc_auc_score(Y_test, Y_pred_proba[:, 1])
accuracy_value = accuracy_score(Y_test, Y_pred)
balanced_accuracy_value = balanced_accuracy_score(Y_test, Y_pred)

# Create results dictionary
test_metrics = {
    'negative_log_loss': neg_log_loss_value,
    'roc_auc': roc_auc_value,
    'accuracy': accuracy_value,
    'balanced_accuracy': balanced_accuracy_value
}

# Display results
print("Test set performance metrics:")
print(test_metrics)

Test set performance metrics:
{'negative_log_loss': 0.5228202702042561, 'roc_auc': 0.6765806883942665, 'accuracy': 0.7629235336267786, 'balanced_accuracy': 0.500163875792931}


c:\Users\Q\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0, 1, 2, 3, 4, 5, 7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Q\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0, 1, 2, 3, 4, 5, 7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

1/ Simplifies Data Processing: By converting the income column to a binary format (0 and 1), it becomes easier to work with the data for ML models. Most algorithms require numerical input, so having income as 0 (for less than or equal to $50K) and 1 (for greater than $50K) simplifies the data preparation process

2/ Clarity: Recoding the target variable right after loading the data provides clarity on what the target variable represents. The code is easier to understand, i.e. now that income is now a binary variable 

3/ Consistency: When evaluating model performance, having a binary target variable allows for straightforward calculations of metrics like accuracy, precision, recall, and F1 score, which enables the metrics to become easier to compute and interpret when the target variable is in a binary format.

4/ Facilitates Data Exploration: With the target variable recoded, it becomes more efficient to explore relationships between features and the target. Creating visualizations or perform statistical analyses becomes easier to see how different features relate to income levels.


## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.